In [25]:
# -*- coding: utf-8 -*-
from tkinter import *
from time import *
import threading
import random

In [26]:
from qiskit import *

In [27]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

In [80]:
def rgbtohex(r,g,b):
    return f'#{r:02x}{g:02x}{b:02x}'

In [81]:
#number of successful whacks
whacks = 0
#number of the hole whacked, no longer needed
hole = 0
qc = QuantumCircuit(30, 30)

#for cx gate
targetsList = []
controlsList = []

gate = -1
isControl = True

#screen elements
screen = Tk()
screen.config(bg="#62C370", width=500, height=520)
screen.resizable(0, 0)
screen.attributes('-topmost', True)
tip = Label(bg='steel blue', fg='green', text="Play with mouse for best experience")
tip.place(x=0, y=497)
lab = Label(text="WHAC-A-MOLE!", font=("Algerian", 30), bg='#3DA5D9', fg='#fff')
lab.place(x=110, y=0)
scorelabel = Label(text=whacks, width=7, font=('Bahnschrift', 30), bg='#FEC601')
scorelabel.place(x=170, y=52)
remark = Label(text="", width=17, bg='#EA7317', fg='#fff', font="Centaur 20")
remark.place(x=130, y=107)

In [82]:
#side buttons and their methods
def applyH():
    global gate
    gate = 0

def applyX():
    global gate
    gate = 1

def applyCX():
    global gate
    gate = 2

def applyZ():
    global gate
    gate = 3

hGate = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='h-gate',
    state='normal',
    command=applyH)
hGate.place(x = 421, y =160)

xGate = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='x-gate',
    state='normal',
    command=applyX)
xGate.place(x = 421, y =225)

cnotGate = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='CNOT',
    state='normal',
    command=applyCX)
cnotGate.place(x = 421, y =290)

zGate = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='z-gate',
    state='normal',
    command=applyZ)
zGate.place(x = 421, y =355)

In [83]:
#runs the game for 60 seconds after start is pressed
def whac_a_mole():
    ready_set_whack()
    global hole
    for i in range(0, 60):
        sleep(1)
    replaybtn.config(state='normal')
    global whacks
    if whacks < 5:
        remark.config(text="POOR")
    elif whacks >= 10 and whacks < 15:
        remark.config(text="BAD")
    elif whacks >= 15 and whacks < 20:
        remark.config(text="DO BETTER!")
    elif whacks >= 25 and whacks < 30:
        remark.config(text="GOOD")
    elif whacks >= 30 and whacks < 35:
        remark.config(text="GREAT!")
    elif whacks >= 35 and whacks < 40:
        remark.config(text="AWESOME!")
    elif whacks == 45:
        remark.config(text="WHACKED 'em ALL!", font="Centaur 18")

#when start button is pressed, whac_a_mole() is run in a separate thread
def start():
    global whacks
    whacks = 0
    global gate
    gate = -1
    
    remark.config(text="")
    replaybtn.config(state='disabled')
    hGate.config(state="disabled")
    xGate.config(state="disabled")
    zGate.config(state="disabled")
    cnotGate.config(state="disabled")
    t = threading.Thread(target=whac_a_mole)
    t.start()

startButton = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='start!',
    state='normal',
    command=start)
startButton.place(x=19, y=90)

#configures the replay button
replaybtn = Button(text="Play Again", command=start)
replaybtn.place(x=0, y=2)

In [84]:
#creates the array of buttons (moles/qubits)

xPos = [19, 86, 153, 220, 287, 354]
#421
yPos = [160, 225, 290, 355, 419]

buttons = []

for xP in xPos:
    for yP in yPos:
        buttons.append(Button(
            width=6,
            height=3,
            bg=rgbtohex(255, 255, 255),
            text='【•】__【•】',
            state='normal',
            command=lambda m=len(buttons): onwhack(m)))
        buttons[len(buttons) - 1].place(x = xP, y = yP)

In [85]:
#event triggered when mole is clicked

def onwhack(num):
    global gate
    if gate >= 0:
        placeGate(num, gate)
    else:
        global hole
        global whacks
        if quantumMeasure(num) == 1:
            whacks += 1
            scorelabel.config(text="* " + str(whacks) + " *")
            buttons[num].config(state='disabled', text='x_x', fg='black')
        elif quantumMeasure(num) == 0:
            buttons[num].config(state='disabled', text='◉‿◉', fg='black')

In [86]:
#stores format of mole face
gateList = ["【•】__【•】\n"] * 30
target = -1
control = -1

#num: the number of the button
#g: same as the global variable gate
def placeGate(num, g):
    global target
    global control 
    global gateList
    global isControl 
    #H gate
    if g == 0:
        qc.h(num)
        gateList[num] = gateList[num] + "H "
        buttons[num].config(bg='#80e8ff', text=gateList[num])
    #X gate
    if g == 1:
        qc.x(num)
        gateList[num] = gateList[num] + "X "
        buttons[num].config(bg='#00d1ff', text=gateList[num])
    #CX gate
    if g == 2:
        if isControl == True:
            control = num
            controlsList.append(num)
            gateList[num] = gateList[num] + "CX:C" + str(len(controlsList)) + ""
            buttons[num].config(bg='#ccf6ff', text=gateList[num])
            isControl = False
        else:
            target = num
            qc.cx(control, target)
            targetsList.append(num)
            gateList[num] = gateList[num] + "CX:T" + str(len(targetsList)) + ""
            buttons[num].config(bg='#ccf6ff', text=gateList[num])
            isControl = True
        print(targetsList)
        print(controlsList)
    #Z gate
    if g== 3:
        gateList[num] = gateList[num] + "Z "
        buttons[num].config(bg='#e6faff', text=gateList[num])

In [87]:
def placeCX(c, t): 
    pair = {
        "control": c, 
        "target": t
    }
    

In [88]:
#happens when mole is clicked outside of place gate time
def quantumMeasure(bnum):
    qc.measure(range(30), range(30))
    sim = Aer.get_backend('qasm_simulator')
    qexe = transpile(qc, sim)
    job = sim.run(qexe, shots = 1)
    result = job.result()
    counts = result.get_counts()
    numberList = list(counts)
    number = numberList[0]
    bit = number[30 - bnum - 1]
    return int(bit)
    #buttons[bnum].config(bg=rgbtohex(255, 255*col, 255))
    

In [89]:
#the message banner changes after start button is clicked
def ready_set_whack():
    sleep(1)
    scorelabel.config(text="Ready")
    sleep(1)
    scorelabel.config(text="Set")
    sleep(1)
    scorelabel.config(text="WHACK!!")
    sleep(1)
    scorelabel.config(text="0")
    for b in buttons: 
        b.config(state='normal')

In [90]:

screen.mainloop()


[]
[15]
[20]
[15]
[20]
[15, 20]
[20, 21]
[15, 20]
[20, 21]
[15, 20, 21]
[20, 21, 26]
[15, 20, 21]
[20, 21, 26]
[15, 20, 21, 26]
[20, 21, 26, 27]
[15, 20, 21, 26]


Exception in thread Thread-6 (whac_a_mole):
Traceback (most recent call last):
  File "C:\Users\emily\anaconda3\envs\condaenvqiskit\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\emily\anaconda3\envs\condaenvqiskit\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\emily\AppData\Local\Temp\ipykernel_1176\3619157663.py", line 7, in whac_a_mole
  File "C:\Users\emily\anaconda3\envs\condaenvqiskit\Lib\tkinter\__init__.py", line 1702, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\emily\anaconda3\envs\condaenvqiskit\Lib\tkinter\__init__.py", line 1692, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
RuntimeError: main thread is not in main loop
